In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

In [4]:
# import fastai 
# from fastai.vision import *

In [5]:
import numpy as np
import datetime
import os, sys

In [6]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow, imsave

In [7]:
from pandas import read_csv
import pandas as pd

"""
df = pd.read_csv('./inputs/dummy-class/t001.eps', sep=' ',
                 header=None, 
                 dtype={'0': 'int', '5': 'string'},
                 skiprows=21,
                 nrows=5000)
print(df.shape)
print(df.dtypes)
print(df[[0]].min())
print(df[[1]].min())
print(df[[3]].min())
print(df[[0]].max())
print(df[[1]].max())
print(df[[3]].max())
"""

In [8]:
eps_max = 590
eps_rows = 500

I thought I would need to save the bounding box and other header info, but I can just calculate that bounding box dynamically based on mins and maxs. 

"""
F = open('workfile,eps','w')
F.write('%!PS-Adobe-3.0 EPSF-3.0\n')
F.write('%%BoundingBox: -90 -32 553 333\n\n')
F.write('/bd { bind def } bind def\n')
F.write('/co { setrgbcolor } bd\n')
F.write('/lw { setlinewidth } bd\n')
F.write('/st { stroke } bd\n')
F.write('/fi { fill } bd\n')
F.write('/cp { closepath } bd\n')
F.write('/mo { newpath moveto } bd\n')
F.write('/to { lineto } bd\n')
F.write('/li { mo to st } bd\n')
F.write('/ci { newpath 0 360 arc } bd\n')
F.write('gsave\n')
F.write('1 lw\n')
F.write('2 setlinecap\n')
F.write('0 setlinejoin\n')
F.write('0.078431 0.588235 0.823529 co\n\n')
F.writelines(df.to_string(header=False, index=False)) 
F.close() 
"""

In [9]:
torch.cuda.empty_cache()

In [10]:
MODEL_NAME = 'VanillaGAN'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DEVICE = "cpu"

In [11]:
print('Using device:', DEVICE)
print()

#Additional Info when using cuda
if hasattr(DEVICE, 'type') and DEVICE.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce GTX 1050
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [12]:
image_cols=4
batch_size = 5

In [13]:
def normalize_to_image(y_hat):
    result = y_hat.mul(eps_max).cpu().int().data.numpy()
    return result

In [14]:
# could not broadcast input array from shape (500,5000) into shape (500,2500)
# could not broadcast input array from shape (250,1250) into shape (250,2500)
# could not broadcast input array from shape (250,1250) into shape (250,2500)

def get_sample_image(G, n_noise):
    """
        generate 5 images
    """
    num_images = 5
    z = torch.randn(num_images, n_noise).to(DEVICE)
    # print(z[0][0:100])
    y_hat = G(z).view(num_images, eps_rows, image_cols)
    result = y_hat.mul(eps_max).cpu().int().data.numpy()
    # result = result.view(25, image_rows, image_cols)
    # print(result[0])
    # print(result[0].shape)
    return result

    img = np.zeros([image_size * 5, image_size * 5])
    for j in range(5):
        # b = np.concatenate([np.where(x > 0.5, 1, 0) for x in result[j*5:(j+1)*5]], axis=-1)
        b = np.concatenate([x for x in result[j*5:(j+1)*5]], axis=-1)
        img[j*image_size:(j+1)*image_size] = b
    return img

[[[-22   6  24 -22]
  [ 23  -6  47  19]
  [-67  29  -5  -6]
  ...
  [-13 -21 -26 -15]
  [-19 -26 -51  26]
  [-35 -22  19   7]]
  
  

```
img = get_sample_image(G, n_noise)
# print(img.shape)
# print(img)
img_lines = img[0]
print(img_lines.shape)


for j in range(image_rows):
    # print(img_lines[j])
    print('{} {} {} {} li\n'.format(img_lines[j][0], img_lines[j][1], img_lines[j][2], img_lines[j][3]))
    
    
# img_lines_str = ' '.join(str(y) + ' li\n' for y in x for x in img_lines)
print(img_lines_str)
# a_str = ' '.join(str(y) + ' li\n' for y in x for x in img[0])

# print(a_str)
```


In [15]:
# Bottom left width. Make this number smaller (more negative) to increase the starting offset from the left
bound_w_min = -320
# Bottom left height. Make this number smaller to increase the height of the image.
bound_h_min = 420
# Top left width. Make this number larger to increase the width of the image.
bound_w_max = 100
# Top left height. Make this number smaller to increase the offset from the top of the image.
bound_h_max = 590

In [16]:
def eps_save(path, img):
    # img = get_sample_image(G, n_noise)
    # print(path)
    F = open(path,'w')
    F.write('%!PS-Adobe-3.0 EPSF-3.0\n')
    F.write('%%BoundingBox: {} {} {} {}\n\n\n\n'.format(bound_w_min, bound_h_min, bound_w_max, bound_h_max,))
    F.write('/bd { bind def } bind def\n')
    F.write('/co { setrgbcolor } bd\n')
    F.write('/lw { setlinewidth } bd\n')
    F.write('/st { stroke } bd\n')
    F.write('/fi { fill } bd\n')
    F.write('/cp { closepath } bd\n')
    F.write('/mo { newpath moveto } bd\n')
    F.write('/to { lineto } bd\n')
    F.write('/li { mo to st } bd\n')
    F.write('/ci { newpath 0 360 arc } bd\n')
    F.write('gsave\n')
    F.write('1 lw\n')
    F.write('2 setlinecap\n')
    F.write('0 setlinejoin\n')
    F.write('0 0 0 co\n')
    
    for j in range(eps_rows):
         # print(img[j][0])
         F.write('{} {} {} {} li\n'.format(img[j][0], img[j][1], img[j][2], img[j][3]))
    
    F.write('grestore\n')
    F.close()

In [17]:
def eps_save_series(step, images):
    for i in range(len(images)): 
        eps_save('./samples/{}_step{:07d}_{}.eps'.format(MODEL_NAME, step, i), images[i])

In [18]:
class Discriminator(nn.Module):
    """
        Simple Discriminator w/ MLP
    """
    def __init__(self, input_size=eps_rows * image_cols, num_classes=1):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 1024),
            # nn.Linear(input_size, 256),            
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            # nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(512, num_classes),
            # nn.Linear(128, num_classes),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        y_ = x.view(x.size(0), -1)
        y_ = self.layer(y_)
        return y_

In [19]:
class Generator(nn.Module):
    """
        Simple Generator w/ MLP
    """
    def __init__(self, input_size=batch_size, num_classes=eps_rows * image_cols):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(num_classes, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            # nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            # nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, num_classes),
            nn.Tanh()
        )
        
    def forward(self, x):
        y_ = self.layer(x)
        y_ = y_.view(x.size(0), 1, eps_rows, image_cols)
        return y_

In [20]:
n_noise = eps_rows * image_cols

In [21]:
def load_one_eps(item_path):
    # print(item_path)
    df = pd.read_csv(item_path, sep=' ',
                 header=None, 
                 # names=['x1', 'y1', 'x2', 'y2', 'cmd'],
                 dtype={0: 'int64', 1: 'int64', 2: 'int64', 3: 'int64', 4: 'string'},
                 skiprows=20,
                 nrows=eps_rows)
    # print(df.loc[:,0:3])
    
    # TODO: Make this into a transform?
    tensor = torch.from_numpy(df.loc[:,0:3].values).float().div(eps_max)
    # img = torch.from_numpy(np.array(pic, np.int16, copy=False))
    # tensor = torch.from_numpy(np.array(df.loc[:,0:3].values, np.uint8, copy=False)).float().div(338)
    return tensor

In [22]:

def load_dataset():
    data_path = 'inputs'
    train_dataset = torchvision.datasets.DatasetFolder(
        root=data_path,
        loader=load_one_eps,
        extensions='eps',
        transform=transforms.Compose([
        #    torchvision.transforms.ToTensor(),
        #    torchvision.transforms.Grayscale(num_output_channels=1),
        #    torchvision.transforms.RandomHorizontalFlip(),
        #    torchvision.transforms.RandomVerticalFlip(),
        #    torchvision.transforms.RandomResizedCrop(248, scale=(0.8, 1.), ratio=(1., 1.)),
        #    # tochvision.transforms.RandomAffine(0, scale=(1.,1.8), fillcolor=255, shear=None),
        #    torchvision.transforms.ToTensor(),
         #   transforms.Normalize(mean=[0.5], std=[0.5])
        ])
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )
    return train_loader

In [23]:
# Show a batch of sample images.
imgs, _ = next(iter(load_dataset()))
print('Batch shape:',imgs.numpy().shape)
# print(imgs.min())
# print(imgs)



Batch shape: (5, 500, 4)


In [24]:
max_epoch = 100000 # need more than 10 epochs for training generator
n_critic = 1 # for training more k steps about Discriminator

In [25]:
D_labels = torch.ones(batch_size, 1).to(DEVICE) # Discriminator Label to real
D_fakes = torch.zeros(batch_size, 1).to(DEVICE) # Discriminator Label to fake

In [26]:
if not os.path.exists('samples'):
    os.makedirs('samples')

In [27]:
def save_checkpoint(state, file_name='checkpoint.pth.tar'):
    torch.save(state, file_name)

In [28]:
torch.cuda.empty_cache()
D = Discriminator().to(DEVICE)
G = Generator(n_noise).to(DEVICE)

In [29]:
#criterion = nn.BCELoss()
#D_opt = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
#G_opt = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

criterion = nn.BCELoss()
D_opt = torch.optim.AdamW(D.parameters(), lr=0.0001, betas=(0.4, 0.999))
G_opt = torch.optim.AdamW(G.parameters(), lr=0.0001, betas=(0.4, 0.999))

In [30]:
step = 1400000

In [33]:
for epoch in range(max_epoch):
    for idx, (images, _) in enumerate(load_dataset()):
        # Training Discriminator
        x = images.to(DEVICE)
        
        x_outputs = D(x)
        # print('x_outputs')
        # print(x_outputs)
        D_x_loss = criterion(x_outputs, D_labels)

        z = torch.randn(batch_size, n_noise).to(DEVICE)
        z_outputs = D(G(z))
        D_z_loss = criterion(z_outputs, D_fakes)
        D_loss = D_x_loss + D_z_loss
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()

        if step % n_critic == 0:
            # Training Generator
            z = torch.randn(batch_size, n_noise).to(DEVICE)
            z_outputs = D(G(z))
            G_loss = criterion(z_outputs, D_labels)

            G.zero_grad()
            G_loss.backward()
            G_opt.step()
            
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, D_loss.item(), G_loss.item()))
        
        if step % 2000 == 0:
            save_checkpoint({'global_step': step,
                'D':D.state_dict(),
                'G':G.state_dict(),
                'd_optim': D_opt.state_dict(),
                'g_optim' : G_opt.state_dict()},
                'ckpt/{}_{:07d}.pth.tar'.format(MODEL_NAME, step))
            G.eval()
            images = get_sample_image(G, n_noise)
            eps_save_series(step, images)
            # imsave('samples/{}_step{}.jpg'.format(MODEL_NAME, str(step).zfill(3)), img, cmap='gray')
            G.train()
        step += 1

Epoch: 0/100000, Step: 1600000, D Loss: 1.1684578657150269, G Loss: 1.423330545425415
Epoch: 500/100000, Step: 1601000, D Loss: 0.9472920894622803, G Loss: 1.2158310413360596
Epoch: 1000/100000, Step: 1602000, D Loss: 1.083784818649292, G Loss: 1.1252762079238892
Epoch: 1500/100000, Step: 1603000, D Loss: 0.6921959519386292, G Loss: 0.89454585313797
Epoch: 2000/100000, Step: 1604000, D Loss: 0.7355842590332031, G Loss: 1.1757228374481201
Epoch: 2500/100000, Step: 1605000, D Loss: 0.8673501014709473, G Loss: 1.217210054397583
Epoch: 3000/100000, Step: 1606000, D Loss: 0.9523478746414185, G Loss: 1.2137908935546875
Epoch: 3500/100000, Step: 1607000, D Loss: 0.8824858665466309, G Loss: 1.2675198316574097
Epoch: 4000/100000, Step: 1608000, D Loss: 0.9947152137756348, G Loss: 1.1962215900421143
Epoch: 4500/100000, Step: 1609000, D Loss: 0.8997101783752441, G Loss: 1.1791092157363892
Epoch: 5000/100000, Step: 1610000, D Loss: 1.0954169034957886, G Loss: 1.2841975688934326
Epoch: 5500/100000,

In [35]:
save_checkpoint({'global_step': step,
                'D':D.state_dict(),
                'G':G.state_dict(),
                'd_optim': D_opt.state_dict(),
                'g_optim' : G_opt.state_dict()},
                'ckpt/{}_{:06d}.pth.tar'.format(MODEL_NAME, step))

# Restore Checkpoint

In [31]:
# import glob
# G_path = sorted(glob.glob(os.path.join('ckpt', 'began080000-good.pth.tar')))[0]
# print(G_path)

# /home/tc/ws/geo-to-picture/ckpt/VanillaGAN_1398000.pth.tar

state = torch.load('ckpt/VanillaGAN_1398000.pth.tar')
# print(state['G'])
G.load_state_dict(state['G'])
D.load_state_dict(state['D'])

<All keys matched successfully>

In [507]:
x = images.to(DEVICE)
n_images = normalize_to_image(x)
eps_save_series(3000, n_images)


In [503]:
G.eval()
# plt.figure(figsize = (15,15))
# plt.imshow(get_sample_image(G, n_noise), cmap='gray')
G(z).view(2, eps_rows, image_cols)

RuntimeError: shape '[2, 500, 4]' is invalid for input of size 10000

In [ ]:
# torch.save(D.state_dict(), 'D.pkl')
# torch.save(G.state_dict(), 'G.pkl')